In [30]:
from pulp import *

# Sets and Indices
months = range(6)
oils = ['VEG1', 'VEG2', 'OIL1', 'OIL2', 'OIL3']
veg_oils = ['VEG1', 'VEG2']
nonveg_oils = ['OIL1', 'OIL2', 'OIL3']

In [31]:
# Data Parameters
prices = {
    'VEG1': [110, 130, 110, 120, 100, 90],
    'VEG2': [120, 130, 140, 110, 120, 100],
    'OIL1': [130, 110, 130, 120, 150, 140],
    'OIL2': [110, 90, 100, 120, 110, 80],
    'OIL3': [115, 115, 95, 125, 105, 135]
}

hardness = {
    'VEG1': 8.8,
    'VEG2': 6.1,
    'OIL1': 2.0,
    'OIL2': 4.2,
    'OIL3': 5.0
}

storage_cost = 5
selling_price = 150
init_stock = 500
final_stock = 500
max_veg = 200
max_nonveg = 250
max_storage = 1000

In [32]:
# Model Definition
model = LpProblem("Food_Manufacture_1", LpMaximize)

# Decision Variables

buy = LpVariable.dicts("Buy", [(o, m) for o in oils for m in months], lowBound=0)
use = LpVariable.dicts("Use", [(o, m) for o in oils for m in months], lowBound=0)
store = LpVariable.dicts("Store", [(o, m) for o in oils for m in months], lowBound=0)
prod = LpVariable.dicts("Prod", months, lowBound=0)

In [33]:
# Objective Function: Maximize Profit
model += (
    lpSum(selling_price * prod[m] for m in months)  # Revenue from production
    - lpSum(prices[o][m] * buy[(o, m)] for o in oils for m in months)  # Cost of buying oils
    - lpSum(storage_cost * store[(o, m)] for o in oils for m in months)  # Storage costs
), "Total_Profit"


In [34]:

# Constraints
# Inventory Balance Constraints
for o in oils:
    for m in months:
        if m == 0:
            model += store[(o, m)] == init_stock + buy[(o, m)] - use[(o, m)], f"Inventory_Balance_{o}_Month_{m}"
        else:
            model += store[(o, m)] == store[(o, m-1)] + buy[(o, m)] - use[(o, m)], f"Inventory_Balance_{o}_Month_{m}"

# Final Stock Constraint (ensure minimum stock at last month)
for o in oils:
    model += store[(o, months[-1])] >= final_stock, f"Final_Stock_{o}"

# Storage Capacity Constraint (limit max storage per oil and month)
for o in oils:
    for m in months:
        model += store[(o, m)] <= max_storage, f"Max_Storage_{o}_Month_{m}"

# Production Capacity Constraints for Veg and Non-Veg Oils
for m in months:
    model += lpSum(use[(o, m)] for o in veg_oils) <= max_veg, f"Max_Veg_Use_Month_{m}"
    model += lpSum(use[(o, m)] for o in nonveg_oils) <= max_nonveg, f"Max_NonVeg_Use_Month_{m}"

# Production Definition (production equals total use of oils)
for m in months:
    model += prod[m] == lpSum(use[(o, m)] for o in oils), f"Production_Definition_Month_{m}"

# Hardness Constraints (average hardness within [3, 6])
for m in months:
    hardness_sum = lpSum(hardness[o] * use[(o, m)] for o in oils)
    model += hardness_sum >= 3 * prod[m], f"Min_Hardness_Month_{m}"
    model += hardness_sum <= 6 * prod[m], f"Max_Hardness_Month_{m}"

In [35]:
# Solve Model
model.solve()


# Output Results
print("Status:", LpStatus[model.status])
print(f"Total Profit: {value(model.objective):.2f}")

for m in months:
    print(f"\nMonth {m + 1}:")
    print(f"  Production = {prod[m].varValue:.2f} tons")
    for o in oils:
        print(f"  {o}: Buy = {buy[(o, m)].varValue:.2f}, Use = {use[(o, m)].varValue:.2f}, Store = {store[(o, m)].varValue:.2f}")

Status: Optimal
Total Profit: 107842.59

Month 1:
  Production = 450.00 tons
  VEG1: Buy = 0.00, Use = 159.26, Store = 340.74
  VEG2: Buy = 0.00, Use = 40.74, Store = 459.26
  OIL1: Buy = 0.00, Use = 0.00, Store = 500.00
  OIL2: Buy = 0.00, Use = 250.00, Store = 250.00
  OIL3: Buy = 0.00, Use = 0.00, Store = 500.00

Month 2:
  Production = 450.00 tons
  VEG1: Buy = 0.00, Use = 159.26, Store = 181.48
  VEG2: Buy = 0.00, Use = 40.74, Store = 418.52
  OIL1: Buy = 0.00, Use = 0.00, Store = 500.00
  OIL2: Buy = 0.00, Use = 250.00, Store = 0.00
  OIL3: Buy = 0.00, Use = 0.00, Store = 500.00

Month 3:
  Production = 450.00 tons
  VEG1: Buy = 0.00, Use = 85.19, Store = 96.30
  VEG2: Buy = 0.00, Use = 114.81, Store = 303.70
  OIL1: Buy = 0.00, Use = 0.00, Store = 500.00
  OIL2: Buy = 0.00, Use = 0.00, Store = 0.00
  OIL3: Buy = 0.00, Use = 250.00, Store = 250.00

Month 4:
  Production = 450.00 tons
  VEG1: Buy = 0.00, Use = 11.11, Store = 85.19
  VEG2: Buy = 0.00, Use = 188.89, Store = 114.81
 